# core

> Dockerfile generation, image building, container running, and testing

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os, re, json, subprocess
from pathlib import Path
from functools import partial
from fastcore.all import listify, joins, is_listy, L, patch, true, concat

## Dockerfile Instructions

`instr` creates a Dockerfile instruction string from a keyword and value. Factory functions (`from_`, `run_`, `cmd_`, etc.) wrap `instr` for each Dockerfile keyword, handling formatting details like tag joining, JSON exec form, and multi-command chaining.

In [ ]:
#| export
def _instr(kw, v): return f'{kw} {v}'

In [ ]:
assert _instr('RUN', 'echo hello') == 'RUN echo hello'

### Instruction factory functions

Each function maps to a Dockerfile keyword with a trailing `_` to avoid clashing with Python builtins.

In [ ]:
#| export
def _from(image:str, tag:str=None, as_:str=None) -> str:
    'From instruction -- base image with optional tag and alias'
    return _instr('FROM', '%s%s%s' % (image, ':%s' % tag if tag else '', ' AS %s' % as_ if as_ else ''))

In [ ]:
assert _from('python', '3.11') == 'FROM python:3.11'
assert _from('ubuntu', as_='builder') == 'FROM ubuntu AS builder'
assert _from('alpine') == 'FROM alpine'

In [ ]:
#| export
def _run(cmd: list | str):
    'RUN instruction -- execute command(s) in shell'
    return _instr('RUN', joins(' && ', listify(cmd)))

In [ ]:
assert _run('apt-get update') == 'RUN apt-get update'
r = _run(['apt-get update', 'apt-get install -y curl'])
assert 'apt-get update && ' in r
assert 'apt-get install -y curl' in r

In [ ]:
#| export
def _apt_install(*pkgs, y=False):
    'RUN apt-get update && apt-get install packages'
    flag = '-y ' if y else ''
    return _run(['apt-get update', f'apt-get install {flag}{" ".join(pkgs)}'])

In [ ]:
assert _apt_install('curl', 'wget', y=True) == 'RUN apt-get update && apt-get install -y curl wget'
assert _apt_install('git') == 'RUN apt-get update && apt-get install git'

In [ ]:
#| export
def _cmd(cmd: list | str):
    "CMD instruction -- default command for container"
    return _instr('CMD', json.dumps(cmd) if is_listy(cmd) else cmd)

In [ ]:
assert _cmd(['python', 'app.py']) == 'CMD ["python", "app.py"]'
assert _cmd('echo hello') == 'CMD echo hello'

In [ ]:
#| export
def _copy(src:str, dst:str, from_:str=None, link=False):
    'COPY instruction -- copy files into image'
    flags = f'{"--link " if link else ""}{"--from=%s " % from_ if from_ else ""}'
    return _instr('COPY', f'{flags}{src} {dst}')

In [ ]:
assert _copy('.', '/app') == 'COPY . /app'
assert _copy('/build/out', '/app', from_='builder') == 'COPY --from=builder /build/out /app'
assert _copy('app/', '.', link=True) == 'COPY --link app/ .'
assert _copy('/app', '/app', from_='builder', link=True) == 'COPY --link --from=builder /app /app'

In [ ]:
#| export
def _add(src:str, dst:str):
    'ADD instruction -- copy files/URLs into image'
    return _instr('ADD', f'{src} {dst}')

In [ ]:
#| export
def _workdir(path):
    'WORKDIR instruction -- set working directory'
    return _instr('WORKDIR', path)

In [ ]:
assert _workdir('/app') == 'WORKDIR /app'

In [ ]:
#| export
def _env(k, v=None):
    'ENV instruction -- set environment variable'
    return _instr('ENV', f'{k}{'=%s' % v if v else ''}')

In [ ]:
assert _env('PATH', '/usr/local/bin') == 'ENV PATH=/usr/local/bin'
assert _env('DEBIAN_FRONTEND=noninteractive') == 'ENV DEBIAN_FRONTEND=noninteractive'

In [ ]:
#| export
def _expose(port):
    'EXPOSE instruction -- declare container port'
    return _instr('EXPOSE', str(port))

In [ ]:
assert _expose(8080) == 'EXPOSE 8080'

In [ ]:
#| export
def _entrypoint(cmd):
    'ENTRYPOINT instruction -- container entrypoint'
    return _instr('ENTRYPOINT', json.dumps(cmd) if is_listy(cmd) else cmd)

In [ ]:
assert _entrypoint(['python', '-m', 'flask']) == 'ENTRYPOINT ["python", "-m", "flask"]'

In [ ]:
#| export
def _arg(nm, def_=None):
    'ARG instruction -- build-time variable'
    return _instr('ARG', f'{nm}{'=%s' % def_ if def_ else ''}')

In [ ]:
assert _arg('VERSION', '1.0') == 'ARG VERSION=1.0'
assert _arg('VERSION') == 'ARG VERSION'

In [ ]:
#| export
def _label(**kw):
    'LABEL instruction -- image metadata'
    return _instr('LABEL', joins(' ', (f'{k}="{v}"' for k, v in kw.items())))

In [ ]:
assert _label(version='1.0', maintainer='me') == 'LABEL version="1.0" maintainer="me"'

In [ ]:
#| export
def _user(u):
    'USER instruction -- set runtime user'
    return _instr('USER', u)

In [ ]:
#| export
def _volume(path):
    'VOLUME instruction -- declare mount point'
    return _instr('VOLUME', json.dumps(path) if is_listy(path) else path)

In [ ]:
assert _volume('/data') == 'VOLUME /data'
assert _volume(['/data', '/logs']) == 'VOLUME ["/data", "/logs"]'

In [ ]:
#| export
def _shell(cmd):
    'SHELL instruction -- override default shell'
    return _instr('SHELL', json.dumps(cmd))

In [ ]:
assert _shell(['/bin/bash', '-c']) == 'SHELL ["/bin/bash", "-c"]'

In [ ]:
#| export
def _healthcheck(cmd, i=None, t=None, r=None, sp=None):
    'HEALTHCHECK instruction -- container health check'
    o2s = lambda k, v: f'--{k}={v}' if v else ''
    o = ' '.join(filter(None, [o2s('interval',i), o2s('timeout',t), o2s('retries', r), o2s('start-period', sp)]))
    c = json.dumps(cmd) if is_listy(cmd) else cmd
    return _instr('HEALTHCHECK', f'{o} CMD {c}'.strip())

In [ ]:
assert 'CMD curl' in _healthcheck('curl -f http://localhost/', i='30s')
assert _healthcheck('curl localhost', i='30s', t='10s') == 'HEALTHCHECK --interval=30s --timeout=10s CMD curl localhost'
assert _healthcheck('curl localhost') == 'HEALTHCHECK CMD curl localhost'

In [ ]:
#| export
def _stop_sig_(sig):
    'STOPSIGNAL instruction -- set stop signal'
    return _instr('STOPSIGNAL', sig)

In [ ]:
#| export
def _on_build(ins):
    'ONBUILD instruction -- trigger for downstream builds'
    return _instr('ONBUILD', str(ins))

In [ ]:
assert _on_build(_run('echo triggered')) == 'ONBUILD RUN echo triggered'

## Dockerfile Builder

The `Dockerfile` class provides a fluent interface for building Dockerfiles. Start with a base image, chain instruction methods, then render or save.

Each method is one line -- it creates an instruction and appends it, returning `self` for chaining.

In [ ]:
#| export
def _parse(path_or_str: Path | str):
    'Parse Dockerfile text into list of instruction strings'
    t = Path(path_or_str).read_text() if isinstance(path_or_str, Path) else path_or_str
    return L.splitlines(re.sub(r'\\\n\s*', '', t)).filter(lambda l: l.strip() and not l.strip().startswith('#'))

In [ ]:
parsed = _parse("# comment\nFROM python:3.11\nRUN apt-get update && \\\n    apt-get install -y curl\nCOPY . /app")
print(parsed)
assert len(parsed) == 3
assert parsed[0] == 'FROM python:3.11'
assert 'apt-get install -y curl' in parsed[1]

In [ ]:
#| export
class Dockerfile(L):
    'Fluent builder for Dockerfiles'
    def _new(self, items, **kw): return type(self)(items, use_list=None, **kw)
    @classmethod
    def load(cls, path:Path=Path('Dockerfile')): return cls(_parse(Path(path)))
    def from_(self, base, tag=None, as_=None): return self._add(_from(base, tag, as_))
    def _add(self, i): return self._new(self.items + [i])
    def run(self, cmd): return self._add(_run(cmd))
    def cmd(self, cmd): return self._add(_cmd(cmd))
    def copy(self, src, dst, from_=None, link=False): return self._add(_copy(src, dst, from_, link))
    def add(self, src, dst): return self._add(_add(src, dst))
    def workdir(self, path='/app'): return self._add(_workdir(path))
    def env(self, key, value=None): return self._add(_env(key, value))
    def expose(self, port): return self._add(_expose(port))
    def entrypoint(self, cmd): return self._add(_entrypoint(cmd))
    def arg(self, name, default=None): return self._add(_arg(name, default))
    def label(self, **kwargs): return self._add(_label(**kwargs))
    def user(self, user): return self._add(_user(user))
    def volume(self, path): return self._add(_volume(path))
    def shell(self, cmd): return self._add(_shell(cmd))
    def healthcheck(self, cmd, **kw): return self._add(_healthcheck(cmd, **kw))
    def stopsignal(self, signal): return self._add(_stop_sig_(signal))
    def onbuild(self, instruction): return self._add(_on_build(instruction))
    def apt_install(self, *pkgs, y=False): return self._add(_apt_install(*pkgs, y=y))
    def __call__(self, kw, *args, **kwargs):
        'Build a generic Dockerfile instruction: kw ARG1 ARG2 --flag=val --bool-flag'
        flags = [f'--{k.rstrip("_").replace("_","-")}={v}' for k,v in kwargs.items() if v not in (True, False, None)]
        flags += [f'--{k.rstrip("_").replace("_","-")}' for k,v in kwargs.items() if v is True]
        return self._add(f'{kw} {" ".join([*flags, *[str(a) for a in args]])}')
    def __getattr__(self, nm):
        'Dispatch unknown instruction names: df.some_instr(arg) → SOME-INSTR arg'
        if nm.startswith('_'): raise AttributeError(nm)
        return partial(self, nm.upper().rstrip('_'))
    def __str__(self): return chr(10).join(self)
    def __repr__(self): return str(self)
    def save(self, path:Path=Path('Dockerfile')):
        Path(path).mk_write(str(self))
        return path

In [ ]:
df = (Dockerfile().from_('python:3.11-slim')
    .run('pip install flask')
    .copy('.', '/app')
    .workdir('/app')
    .expose(5000)
    .cmd(['python', 'app.py']))

expected = """FROM python:3.11-slim
RUN pip install flask
COPY . /app
WORKDIR /app
EXPOSE 5000
CMD [\"python\", \"app.py\"]"""

assert str(df) == expected
print(df)

Multi-stage builds work naturally:

In [ ]:
df = (Dockerfile().from_('golang:1.21', as_='builder')
    .workdir('/src')
    .copy('.', '.')
    .run('go build -o /app')
    .from_('alpine')
    .copy('/app', '/app', from_='builder')
    .cmd(['/app']))

assert 'FROM golang:1.21 AS builder' in str(df)
assert 'COPY --from=builder /app /app' in str(df)
print(df)

Multi-command RUN chains with `&&`:

In [ ]:
df = (Dockerfile().from_('ubuntu:22.04').run(['apt-get update', 'apt-get install -y python3', 'rm -rf /var/lib/apt/lists/*']))
print(df)

### Loading from an existing Dockerfile

Use `Dockerfile.load()` to read an existing Dockerfile. `save()` returns the `Path` it wrote to.

In [ ]:
import tempfile
tmp = tempfile.mkdtemp()
Path(f'{tmp}/Dockerfile').write_text("# My app\nFROM python:3.11-slim\nRUN apt-get update && \\\n    apt-get install -y curl\nCOPY . /app\nCMD [\"python\", \"app.py\"]")

# Load existing Dockerfile
df = Dockerfile.load(f'{tmp}/Dockerfile')
assert len(df) == 4
assert df[0] == 'FROM python:3.11-slim'

# save returns the path and writes the file
p = df.save(f'{tmp}/Dockerfile')
assert Path(p).exists()

# chain after loading
df2 = df.run('echo hi')
assert len(df2) == 5
print(df)

## Build, Run, Test

These top-level functions wrap the Docker CLI for the common workflow: build an image from a `Dockerfile`, run a container, and test that a command succeeds inside an image.

### Requires Docker daemon

The functions below need a running Docker daemon.

In [ ]:
#| export
def _clean_cfg():
    'Create a docker config dir with credential helpers stripped'
    src = Path(os.environ.get('DOCKER_CONFIG', Path.home()/'.docker'))
    dst = Path.home()/'.dockr'/'config'
    cfgf = dst/'config.json'
    if cfgf.exists(): return str(dst)
    cfg = src.joinpath('config.json').read_json() if (src/'config.json').exists() else {}
    cfg.pop('credsStore', None); cfg.pop('credHelpers', None)
    cfgf.write_json(cfg)
    ctx_src, ctx_dst = src/'contexts', dst/'contexts'
    if ctx_src.exists() and not ctx_dst.exists(): ctx_dst.symlink_to(ctx_src)
    return str(dst)

def calldocker(*args, no_creds=False):
    'Run a docker CLI command, return stdout. Respects DOCKR_RUNTIME env var (default: docker).'
    rt = os.environ.get('DOCKR_RUNTIME', 'docker')
    pre = ('--config', _clean_cfg()) if no_creds and rt == 'docker' else ()
    return subprocess.run((rt,) + pre + args, capture_output=True, text=True, check=True).stdout.strip()

class Docker:
    'Wrap docker CLI: __getattr__ dispatches subcommands, kwargs become flags'
    def __init__(self, no_creds=False): self.no_creds = no_creds

    def __call__(self, cmd, *args, **kwargs):
        fargs = list(args)
        fargs += concat([f'-{k}', str(v)] for k,v in kwargs.items() if len(k)==1 and v not in (True, False, None))
        fargs += [f'-{k}' for k,v in kwargs.items() if len(k)==1 and v is True]
        fargs += [f'--{k.rstrip("_").replace("_","-")}={v}' for k,v in kwargs.items() if len(k)>1 and v not in (True, False, None)]
        fargs += [f'--{k.rstrip("_").replace("_","-")}' for k,v in kwargs.items() if len(k)>1 and v is True]
        return calldocker(cmd, *fargs, no_creds=self.no_creds)

    def __getattr__(self, nm):
        if nm.startswith('_'): raise AttributeError(nm)
        return partial(self, nm.replace('_', '-'))

dk = Docker()

In [ ]:
#| export
@patch
def build(df:Dockerfile, tag:str=None, path:str='.', rm=True, no_creds=False):
    'Build image from Dockerfile. path is the build context directory.'
    df.save(Path(path) / 'Dockerfile')
    Docker(no_creds=no_creds).build(str(path), t=tag, rm=rm)
    return tag

In [ ]:
#| export
def test(img_or_tag:str, cmd):
    'Run cmd in image, return True if exit code 0'
    try: dk.run('--rm', str(img_or_tag), *listify(cmd)); return True
    except Exception: return False

In [ ]:
#| export
def run(img_or_tag:str, detach=False, ports=None, name=None, remove=False, command=None):
    'Run a container, return container ID (detached) or output'
    args = []
    if detach: args.append('-d')
    if remove: args.append('--rm')
    if name: args += ['--name', name]
    for cp, hp in (ports or {}).items(): args += ['-p', f'{hp}:{cp.split("/")[0]}']
    return dk('run', *args, str(img_or_tag), *listify(command or []))

### Convenience functions

In [ ]:
#| export
def containers(all=False):
    'List running containers (names)'
    return dk.ps(format='{{.Names}}', a=all).splitlines()

In [ ]:
#| export
def images():
    'List image tags'
    return dk.images(format='{{.Repository}}:{{.Tag}}').splitlines()

In [ ]:
#| export
def stop(name_or_id:str):
    'Stop a container by name or ID'
    dk.stop(name_or_id)

In [ ]:
#| export
def logs(name_or_id:str, n=10):
    'Tail logs of a container'
    return dk.logs(name_or_id, tail=n)

In [ ]:
#| export
def rm(name_or_id:str, force=False):
    'Remove a container by name or ID'
    dk.rm(name_or_id, f=force)

In [ ]:
#| export
def rmi(image:str, force=False):
    'Remove an image by name or ID'
    dk.rmi(image, f=force)

### Example: FastHTML app with uv

A realistic Dockerfile for a [FastHTML](https://fastht.ml) app that uses `uv` for dependency management, installs system packages, and is designed to run with a mounted volume for persistent data.

In [ ]:
df = (Dockerfile().from_('python', '3.12-slim')
    .apt_install('curl', 'sqlite3', y=True)
    .run('pip install uv')
    .workdir('/app')
    .copy('pyproject.toml', '.')
    .run('uv export --no-hashes -o requirements.txt && pip install -r requirements.txt')
    .copy('.', '.')
    .volume('/app/data')
    .expose(5001)
    .cmd(['python', 'main.py']))

print(df)

In [ ]:
import tempfile
tmp = tempfile.mkdtemp()

df = Dockerfile().from_('alpine').run('echo hello > /greeting.txt').cmd(['cat', '/greeting.txt'])
try:
    tag = df.build(tag='dockr-test:hello', path=tmp, no_creds=True)
    print(f'Built: {tag}')
    out = run(tag, remove=True)
    print(f'Output: {out}')
    rmi(tag)
    print('Cleaned up.')
except Exception as e: print(f'Docker not available: {e}')

### End-to-end: FastHTML + FastLite todo app

In [ ]:
import tempfile, os
app_dir = Path(tempfile.mkdtemp()) / 'fasthtml-todo'
app_dir.mkdir()

# --- main.py: FastHTML + FastLite todo app ---
(app_dir / 'main.py').write_text('''import json as jsonlib
from fasthtml.common import *

db = database('data/todos.db')
todos = db.t.todos
if todos not in db.t: todos.create(id=int, title=str, done=bool, pk='id')
Todo = todos.dataclass()

app, rt = fast_app(live=False)

@rt('/')
def get():
    items = [Li(f"{'✓' if t.done else '○'} {t.title}", id=f'todo-{t.id}') for t in todos()]
    return Titled('Todos',
        Ul(*items),
        Form(Input(name='title', placeholder='New todo...'), Button('Add'), action='/add', method='post'))

@rt('/add', methods=['post'])
def post(title: str):
    todos.insert(Todo(title=title, done=False))
    return Redirect('/')

@rt('/api/todos')
def api():
    data = [dict(id=t.id, title=t.title, done=t.done) for t in todos()]
    return Response(jsonlib.dumps(data), media_type='application/json')

serve(host='0.0.0.0', port=5001)
''')

# --- requirements.txt ---
(app_dir / 'requirements.txt').write_text('python-fasthtml\n')

print(f'App dir: {app_dir}')
print('Files:', os.listdir(app_dir))

In [ ]:
df = (Dockerfile()
    .from_('python', '3.12-slim')
    .workdir('/app')
    .copy('requirements.txt', '.')
    .run('pip install --no-cache-dir -r requirements.txt')
    .copy('.', '.')
    .volume('/app/data')
    .expose(5001)
    .cmd(['python', 'main.py']))

print(df)

In [ ]:
import time
from fastcore.net import urlread, urljson

tag = 'dockr-fasthtml:latest'
name = 'dockr-fasthtml-demo'
try:
    df.build(tag=tag, path=str(app_dir), no_creds=True)
    print(f'Built: {tag}')
    try: rm(name, force=True)
    except: pass
    cid = run(tag, detach=True, ports={'5001/tcp': 5001}, name=name)
    print(f'Container: {cid[:12]}')
    time.sleep(3)

    # Add some todos via POST
    url = 'http://localhost:5001'
    for t in ['Buy milk', 'Write docs', 'Ship dockr']: urlread(f'{url}/add', title=t)
    # Fetch the JSON API
    for t in urljson(f'{url}/api/todos'): print(f"  {'✓' if t['done'] else '○'} {t['title']}")
    print(f'\nLogs:')
    print(logs(name, n=3))
except IOError as e: print(f'Docker not available: {e}')
finally:
    try: rm(name, force=True)
    except: pass
    try: rmi(tag)
    except: pass
    print('Cleaned up.')

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()